In [30]:
# Import the dependencies.
import pandas as pd
import numpy as np
from citipy import citipy

In [31]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [32]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [33]:
# Create a set of random latitude and longitude combinations.
# testing batch
# lats = np.random.uniform(low=-90.000, high=90.000, size=20)
# lngs = np.random.uniform(low=-180.000, high=180.000, size=20)
# main 2000 batch
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [34]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [35]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

751

In [36]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":city_description
                          })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | moiyabana
City not found. Skipping...
Processing Record 2 of Set 1 | male
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | bluff
Processing Record 5 of Set 1 | vaini
Processing Record 6 of Set 1 | busselton
Processing Record 7 of Set 1 | merritt island
Processing Record 8 of Set 1 | nouadhibou
Processing Record 9 of Set 1 | sungaipenuh
Processing Record 10 of Set 1 | maragogi
Processing Record 11 of Set 1 | hilo
Processing Record 12 of Set 1 | zhigansk
Processing Record 13 of Set 1 | mabaruma
Processing Record 14 of Set 1 | nakamura
Processing Record 15 of Set 1 | tasiilaq
Processing Record 16 of Set 1 | jamestown
Processing Record 17 of Set 1 | severo-kurilsk
Processing Record 18 of Set 1 | yenagoa
Processing Record 19 of Set 1 | cidreira
Processing Record 20 of Set 1 | yumen
Processing Record 21 of Set 1 | geylegphug
City not found. Skipping...
Processing Record 22 of 

In [37]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Male,MV,4.1748,73.5089,84.15,79,20,17.27,few clouds
1,Punta Arenas,CL,-53.1500,-70.9167,48.31,66,40,31.07,light intensity shower rain
2,Bluff,NZ,-46.6000,168.3333,44.80,85,100,21.97,light rain
3,Vaini,TO,-21.2000,-175.2000,74.03,79,97,21.16,overcast clouds
4,Busselton,AU,-33.6500,115.3333,45.09,60,92,4.09,overcast clouds


In [40]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file)